In [48]:
import sklearn
import pandas as pd
from influxdb import InfluxDBClient
from sklearn import manifold, random_projection
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
from metrics.pyDRMetrics import DRMetrics
import matplotlib.pyplot as plt
import yaml
import time

In [49]:
with open('/home/min/Multi-level-Feature-Driven-Storage-Server/configuration_manager/config.yaml','r') as yamlfile:
    data = yaml.load(yamlfile,Loader=yaml.FullLoader)

DBclient = InfluxDBClient(host=data['global']['database_ip'], port=data['global']['database_port'], database=data['global']['database_name'], username='root', password='root')

max_ = 0
min_ = 0

# Note that the input values to PCA should not be negative or the principle would be different because of the order 
table_name = "visual_features_entropy_unnormalized_arena"
result = DBclient.query('SELECT * FROM '+table_name)
result_point = pd.DataFrame(result.get_points(measurement=table_name))
meta_X = result_point['name']

X = result_point.drop(columns = ['name', 'time', 'base_path', 'storage_path'])



In [50]:
## dimension reduction: isomap
isomap = manifold.Isomap(n_neighbors=12, n_components=1)
isomap_time = time.time()
isomap_y = isomap.fit_transform(X)
isomap_time = time.time()-isomap_time

## dimension reduction: t-SNE
tsne = manifold.TSNE(n_components=1)
tsne_time = time.time()
tsne_y = tsne.fit_transform(X)
tsne_time = time.time()-tsne_time

## dimension reduction: pca
pca = PCA(n_components=1)
pca_time = time.time()
pca_y = pca.fit_transform(X)
pca_time = time.time() - pca_time

## dimension reduction: mds
mds = manifold.MDS(n_components=1)
mds_time = time.time()
mds_y = mds.fit_transform(X)
mds_time = time.time() - mds_time

## dimension reduction: rps
rps_transformer = random_projection.SparseRandomProjection(n_components = 1)
rps_time = time.time()
rps_y = rps_transformer.fit_transform(X)
rps_time = time.time() - rps_time

## dimension reduction: rpg
rpg_transformer = random_projection.GaussianRandomProjection(n_components = 1)
rpg_time = time.time()
rpg_y = rpg_transformer.fit_transform(X)
rpg_time = time.time() - rpg_time

In [51]:
# minMax normalization (isomap)
max_ = max(0, isomap_y.max(axis=0))
min_ = min(0, isomap_y.min(axis=0))
isomap_y_std = (isomap_y - min_) / (max_ - min_)

# minMax normalization (t-SNE)
max_ = max(0, tsne_y.max(axis=0))
min_ = min(0, tsne_y.min(axis=0))
tsne_y_std = (tsne_y - min_) / (max_ - min_)

# minMax normalization (pca)
max_ = max(0, pca_y.max(axis=0))
min_ = min(0, pca_y.min(axis=0))
pca_y_std = (pca_y - min_) / (max_ - min_)

# minMax normalization (mds)
max_ = max(0, mds_y.max(axis=0))
min_ = min(0, mds_y.min(axis=0))
mds_y_std = (mds_y - min_) / (max_ - min_)

# minMax normalization (rps)
max_ = max(0, rps_y.max(axis=0))
min_ = min(0, rps_y.min(axis=0))
rps_y_std = (rps_y - min_) / (max_ - min_)

# minMax normalization (rpg)
max_ = max(0, rpg_y.max(axis=0))
min_ = min(0, rpg_y.min(axis=0))
rpg_y_std = (rpg_y - min_) / (max_ - min_)

drm_isomap = DRMetrics(X, isomap_y_std)
drm_tsne = DRMetrics(X, tsne_y_std)
drm_pca = DRMetrics(X, pca_y_std)
drm_mds = DRMetrics(X, mds_y_std)
drm_rps = DRMetrics(X, rps_y_std)
drm_rpg = DRMetrics(X, rpg_y_std)

In [52]:
# plt.plot(drm_pca.T, label = 'pca', color = 'b')
# plt.plot(drm_isomap.T, label = 'isomap', color = 'y')
# plt.plot(drm_rps.T, label = 'isomap', color = 'g')
# plt.plot(drm_rpg.T, label = 'isomap', color = 'r')

# plt.ylim(-0.1,1.1)
# plt.legend()
# plt.savefig('T.png')
# plt.clf()

# plt.plot(drm_pca.C, label = 'pca', color = 'b')
# plt.plot(drm_isomap.C, label = 'isomap', color = 'y')
# plt.plot(drm_rps.C, label = 'isomap', color = 'g')
# plt.plot(drm_rpg.C, label = 'isomap', color = 'r')

# plt.ylim(-0.1,1.1)
# plt.legend()
# plt.savefig('C.png')
# plt.clf()

print('iso T AUC = ', drm_isomap.AUC_T)
print('tsne T AUC =', drm_tsne.AUC_T)
print('PCA T AUC = ', drm_pca.AUC_T)
print('MDS T AUC = ', drm_mds.AUC_T)
print('rps T AUC = ', drm_rps.AUC_T)
print('rpg T AUC = ', drm_rpg.AUC_T)

print('iso C AUC = ', drm_isomap.AUC_C)
print('tsne C AUC =', drm_tsne.AUC_C)
print('PCA C AUC = ', drm_pca.AUC_C)
print('MDS T AUC = ', drm_mds.AUC_C)
print('rps C AUC = ', drm_rps.AUC_C)
print('rpg C AUC = ', drm_rpg.AUC_C)
print("---------")
print("Time:")
print("isomap:", isomap_time)
print("PCA:", pca_time)
print("MDS:", mds_time)
print("t-SNE", tsne_time)
print("rps:", rps_time)
print("rpg:", rpg_time)

iso T AUC =  0.9999007483982936
tsne T AUC = 0.9685320234937127
PCA T AUC =  0.9999249906621952
MDS T AUC =  0.9622204433411328
rps T AUC =  0.9976373560528395
rpg T AUC =  0.9998175985105616
iso C AUC =  0.9999519418065419
tsne C AUC = 0.9775880428124323
PCA C AUC =  0.999973327470415
MDS T AUC =  0.9208463430658848
rps C AUC =  0.9977832884300278
rpg C AUC =  0.9999030677380142
---------
Time:
isomap: 0.04683375358581543
PCA: 0.0058269500732421875
MDS: 0.29244065284729004
t-SNE 24.49342441558838
rps: 0.008717775344848633
rpg: 0.010856866836547852


In [53]:
print("Pearson Correlation Coefficient:")
print("iso: %.9f"% (drm_isomap.Vr))
print("tsne: %.9f"% (drm_tsne.Vr))
print("pca: %.9f"% (drm_pca.Vr))
print("mds: %.9f"% (drm_mds.Vr))
print("rps: %.9f"% (drm_rps.Vr))
print("rpg: %.9f"% (drm_rpg.Vr))
print("Spearman Correlation Coefficient:")
print("iso: %.9f"% (drm_isomap.Vrs))
print("tsne: %.9f"% (drm_tsne.Vrs))
print("pca: %.9f"% (drm_pca.Vrs))
print("mds: %.9f"% (drm_mds.Vrs))
print("rps: %.9f"% (drm_rps.Vrs))
print("rpg: %.9f"% (drm_rpg.Vrs))

Pearson Correlation Coefficient:
iso: 0.000113554
tsne: 0.525636135
pca: 0.000071070
mds: 0.266755216
rps: 0.009772027
rpg: 0.000380455
Spearman Correlation Coefficient:
iso: 0.000778944
tsne: 0.446718749
pca: 0.000730420
mds: 0.141329128
rps: 0.011943686
rpg: 0.001158974
